In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import graphviz

In [2]:
titles = pd.read_csv('titles.csv')

In [17]:
df = pd.DataFrame(columns=['source', 'target', 'weight'])

for i in range(len(titles)):
    countries = titles['production_countries'][i].strip('[]').split(', ')
    for j in range(len(countries)):
        for k in range(j+1, len(countries)):
            if df[(df['source'] == countries[j]) & (df['target'] == countries[k])].empty:
                df = pd.concat([df, pd.DataFrame({'source': countries[j], 'target': countries[k], 'weight': [1]})], ignore_index=True)
            else:
                df.loc[(df['source'] == countries[j]) & (df['target'] == countries[k]), 'weight'] += 1

In [18]:
df.to_csv('edges.csv', index=False)

In [23]:
from pyvis.network import Network

net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')
net.barnes_hut()

sources = df['source']
targets = df['target']
weights = df['weight']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    net.add_node(src, src, title=src)
    net.add_node(dst, dst, title=dst)
    net.add_edge(src, dst, value=w)
    
neighbor_map = net.get_adj_list()

for node in net.nodes:
    node['value'] = len(neighbor_map[node['id']])

net.write_html('country_network.html')